In [1]:
#Author: @michaelbrink
#Org: BalloonBox Inc.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Helper Functions

In [3]:
def data_type_transform(df_origin):

    # Make a copy
    df = df_origin.copy()

    # The list of text features that don't need to clean
    text_list=['Name','Country','Headquarters','Industry','CEO','Website','Company Type','Ticker']

    # The list of all the features
    feature_list=df.columns
    
    # For loop for cleaninn
    for col in feature_list: 
        # Text data doesn't need to clean
        if col in text_list:
            pass
        else:
            # 1: The '-' value in '%' columns gonna be 0!
            df[col] = df[col].replace(to_replace='[\$,%]',value='',regex=True)

            # 2: The '-' value in ($M) columns will be dropped
            if ('%' in col) or ('Change' in col) or ('Return' in col):
                df[col] = df[col].replace(to_replace='^-$',value='0',regex=True)
            
            # 3: Convert them into numeric values
            df[col] = pd.to_numeric(df[col],errors='coerce')
            
            # # 4: Divide by 100 for all '%' columns
            # if ('%' in col) or ('Return' in col):
            #     df[col] = round(df[col]/100,5)
    return df

In [4]:
def drop_by_quantile(df_origin,cols):
    '''
        df is the dataframe we want to process: DataFrame
        cols is a list of columns we want to process: List
    '''
    # Make a copy of DataFrame
    df = df_origin.copy()

    # List contains the index of dropped rows
    drop_list = []

    for col in cols:
        # drop the outliers based on boxplot
        temp = list(df[col].quantile([0.25,0.75]))
        Q1 = temp[0]
        Q3 = temp[1]
        IQR = Q3-Q1
        minimum = Q1-1.5*IQR
        maximum = Q3+1.5*IQR
        # Append the index of dropped rows
        low_outlier = df[df[col]<=minimum].index
        high_outlier = df[df[col]>=maximum].index
        for index in low_outlier:
            drop_list.append(index)
        for index in high_outlier:
            drop_list.append(index)
        
        # Deduplicated index
        drop_list = list(dict.fromkeys(drop_list))
    
    # Drop the outliers
    df.drop(drop_list,axis=0,inplace=True)

    # # Histogram
    # plt.hist(df['Employees'],bins=5)

    return df

In [5]:
def create_buckets(df_origin,cols):
    # Make a copy of origin dataset
    df = df_origin.copy()
    temp_df = pd.DataFrame()

    # Create 5 equal size buckets
    for col in cols:
        col_value = df[col]
        qcut_value, bins = pd.qcut(x=col_value,q=[0,0.2,0.4,0.6,0.8,1.0],labels=[1,2,3,4,5],retbins=True)
        temp_df[col+'_cat'] = qcut_value
        # print('The bins of {col} is : {bins}'.format(col=col,bins=bins))
    
    # Concat the dataFrame
    df = pd.concat([df,temp_df],axis=1)
    return df

# Imported the data files

In [6]:
# Read in the original datasets
df_all_origin=pd.read_csv('Fortune500.csv')
df_detail_origin=pd.read_csv('Fortune500-2.csv')

# DATA ENGINEERING

## df_all

In [39]:
# Back up the original dataset
df_all = df_all_origin.copy()
print('The shape of original df_all is :',df_all.shape)


# Data type transformation for df_all dataset
df_all = data_type_transform(df_all)


# Drop null values
if df_all.isna().sum().sum() != 0:
    df_all.dropna(inplace=True)
print('The shape of df_all after dropping null value is : ',df_all.shape)


# Drop outliers
drop_cols = ['Profit ($M)','Assets ($M)','Market Value ($M)','Employees']
df_all = drop_by_quantile(df_all,drop_cols)
print('The shape of df_all after dropping outliers is : ',df_all.shape)


# Create three buckets for Revenue(1:Low, 2:Medium, 3:High)
conditions = [df_all['Revenue ($M)']<5000,(df_all['Revenue ($M)']>=5000)&(df_all['Revenue ($M)']<=10000),df_all['Revenue ($M)']>10000]
values = [1,2,3]
df_all['Revenue_cat'] = np.select(conditions,values)


# Combine filter's columns together(0: No filter, 1: Female CEO, 2: Founder CEO, 3: Both)
conditions = [(df_all['Female CEO']==0)&(df_all['Founder CEO']==0),\
              (df_all['Female CEO']==1)&(df_all['Founder CEO']==0),\
              (df_all['Female CEO']==0)&(df_all['Founder CEO']==1),\
              (df_all['Female CEO']==1)&(df_all['Founder CEO']==1)]
values = [0,1,2,3]
df_all['Filter_cat'] = np.select(conditions,values)
if 'Female CEO' or 'Founder CEO' in df_all.columns:
    df_all.drop(columns=['Female CEO','Founder CEO'],inplace=True)


# Create buckets
process_list = ['Profit ($M)','Assets ($M)','Market Value ($M)','Employees']
df_all = create_buckets(df_all,process_list)
print('The shape of df_all after creating buckets is : ',df_all.shape)

# # Create buckets for df_all
# process_list = ['Filter_Cat','Profit ($M)_Cat','Assets ($M)_Cat','Market Value ($M)_Cat','Employees_Cat']
# df_all = dummies(df_all,process_list)
# print('The shape of df_all after getting dummies is : ',df_all.shape)
# df_all

The shape of original df_all is : (4000, 14)
The shape of df_all after dropping null value is :  (3774, 14)
The shape of df_all after dropping outliers is :  (2801, 14)
The shape of df_all after creating buckets is :  (2801, 18)


In [40]:
df_all.columns

Index(['Rank', 'Name', 'Revenue ($M)', 'Revenue % change', 'Profit ($M)',
       'Profit % change', 'Assets ($M)', 'Market Value ($M)',
       'Change in rank (1000)', 'Employees', 'Change in rank (500)', 'Year',
       'Revenue_cat', 'Filter_cat', 'Profit ($M)_cat', 'Assets ($M)_cat',
       'Market Value ($M)_cat', 'Employees_cat'],
      dtype='object')

In [60]:
def dummies(df_origin,cols):
    # Make a copy of origin dataset
    df = df_origin.copy()
    temp_df =pd.get_dummies(df['Profit ($M)_cat'])
    display(temp_df)
    # # Concat the dataFrame
    # df = pd.concat([df,temp_df],axis=1)
    # return df

In [61]:
dummies(df_all,['Filter_cat','Profit ($M)_cat','Assets ($M)_cat','Market Value ($M)_cat','Employees_cat'])

,1,2,3,4,5
10,0,0,0,0,1
14,0,0,0,0,1
33,0,0,0,0,1
36,0,0,0,0,1
44,0,0,0,0,1
...,...,...,...,...,...
3995,0,0,1,0,0
3996,0,1,0,0,0
3997,0,1,0,0,0
3998,1,0,0,0,0


## Data Preprocessing(String to Numerics)

In [ ]:
# firstly, drop the duplicated and useless columns in df_detail
feature_list = df_detail.columns
drop_list = ['Revenues ($M).1','Profits ($M).1','Unnamed: 25']
for feature in feature_list:
    if feature in drop_list:
        df_detail.drop(columns=feature,inplace=True)

In [ ]:
# Data cleaning for df_detail dataset
df_detail_cleaned = data_cleaning(df_detail)

print('The shape of original df_detail is :',df_detail.shape)

# Drop null value
if df_detail_cleaned.isna().sum().sum() != 0:
    df_detail_cleaned.dropna(inplace=True)

print('The shape of cleaned de_detail is : ',df_detail_cleaned.shape)

## Data Cleaning(Drop outilers)

In [ ]:
# Process df_detail_cleaned dataset
drop_cols = ['Profits ($M)','Market Value ($M)','Employees','Assets ($M)',\
             'Total Stockholder Equity ($M)','Earnings Per Share ($)']
df_detail_dropped = drop_by_quantile(df_detail_cleaned,drop_cols)

# Results from each step
print('The shape of original df_all is :',df_detail.shape)
print('The shape of cleaned df_all is : ',df_detail_cleaned.shape)
print('The shape of dropped df_all(quantiles) is : ',df_detail_dropped.shape)

## Data Preprocessing(Numerics to categoricals, equal size buckets)

In [ ]:
# Dummy process for df_detail
process_list = ['Revenues ($M)','Profits ($M)','Market Value ($M)','Employees','Assets ($M)',\
             'Total Stockholder Equity ($M)','Earnings Per Share ($)']
df_detail_cat = create_buckets(df_detail_dropped,process_list)

# Results from each step
print('\n')
print('The shape of original df_all is :',df_detail.shape)
print('The shape of cleaned df_all is : ',df_detail_cleaned.shape)
print('The shape of dropped df_all(quantiles) is : ',df_detail_dropped.shape)
print('The shape of categorized df_all is : ',df_detail_cat.shape)

## get dummies

In [ ]:
# Output the csv files
df_all_final.to_csv('Fortune500_cleaned.csv',index=False)
df_detail_final.to_csv('Fortune500-2_cleaned.csv',index=False)

In [ ]:
df_all_dropped